In [1]:
import requests
import defs
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle("instruments.pkl")

In [4]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD','CAD']

In [5]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [6]:
code, res = fetch_candles("EUR_USD",9,"H1")

In [7]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 5773,
   'time': '2024-09-25T07:00:00.000000000Z',
   'bid': {'o': '1.11899', 'h': '1.11958', 'l': '1.11851', 'c': '1.11911'},
   'mid': {'o': '1.11906', 'h': '1.11966', 'l': '1.11858', 'c': '1.11918'},
   'ask': {'o': '1.11913', 'h': '1.11973', 'l': '1.11866', 'c': '1.11926'}},
  {'complete': True,
   'volume': 5051,
   'time': '2024-09-25T08:00:00.000000000Z',
   'bid': {'o': '1.11906', 'h': '1.11948', 'l': '1.11814', 'c': '1.11841'},
   'mid': {'o': '1.11914', 'h': '1.11956', 'l': '1.11822', 'c': '1.11849'},
   'ask': {'o': '1.11923', 'h': '1.11963', 'l': '1.11829', 'c': '1.11857'}},
  {'complete': True,
   'volume': 4254,
   'time': '2024-09-25T09:00:00.000000000Z',
   'bid': {'o': '1.11841', 'h': '1.11871', 'l': '1.11774', 'c': '1.11825'},
   'mid': {'o': '1.11848', 'h': '1.11878', 'l': '1.11782', 'c': '1.11832'},
   'ask': {'o': '1.11856', 'h': '1.11886', 'l': '1.11790', 'c': '1.11840'}},

In [8]:
code

200

In [9]:
def get_candles_df(json_response):
    
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)


In [10]:
get_candles_df(res)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-09-25T07:00:00.000000000Z,5773,1.11906,1.11966,1.11858,1.11918,1.11899,1.11958,1.11851,1.11911,1.11913,1.11973,1.11866,1.11926
1,2024-09-25T08:00:00.000000000Z,5051,1.11914,1.11956,1.11822,1.11849,1.11906,1.11948,1.11814,1.11841,1.11923,1.11963,1.11829,1.11857
2,2024-09-25T09:00:00.000000000Z,4254,1.11848,1.11878,1.11782,1.11832,1.11841,1.11871,1.11774,1.11825,1.11856,1.11886,1.11790,1.11840
3,2024-09-25T10:00:00.000000000Z,4390,1.11833,1.11890,1.11810,1.11872,1.11826,1.11883,1.11802,1.11864,1.11840,1.11897,1.11817,1.11879
4,2024-09-25T11:00:00.000000000Z,3942,1.11873,1.11974,1.11833,1.11959,1.11866,1.11966,1.11825,1.11951,1.11880,1.11982,1.11841,1.11967
5,2024-09-25T12:00:00.000000000Z,6316,1.11960,1.12079,1.11918,1.11978,1.11952,1.12071,1.11910,1.11969,1.11967,1.12087,1.11926,1.11986
6,2024-09-25T13:00:00.000000000Z,8310,1.11979,1.12142,1.11952,1.11952,1.11971,1.12133,1.11945,1.11945,1.11987,1.12150,1.11960,1.11960
7,2024-09-25T14:00:00.000000000Z,7100,1.11953,1.11956,1.11654,1.11726,1.11945,1.11948,1.11645,1.11718,1.11961,1.11964,1.11661,1.11734


In [11]:
def save_file(candles_df,pair,granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [12]:
def create_data(pair,granularity):
    code ,json_data = fetch_candles(pair,4000,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,pair, granularity)

In [13]:
for p1 in our_curr:
    for p2 in our_curr:
        pair =  f"{p1}_{p2}"
        if pair in ins_df.name.unique():
          create_data(pair,"H1")

EUR_USD loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
USD_JPY loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
USD_CHF loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
USD_CAD loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
GBP_USD loaded 3999 candles from 2024-02-05T23:00:00.000000000Z to 2024-09-25T14:00:00.000000000Z
GBP_JPY loaded 3999 

In [14]:
ins_df.shape[0]

68

In [15]:
ins_df

,name,type,displayName,pipLocation,marginRate
0,USD_CNH,CURRENCY,USD/CNH,-4,0.05
1,NZD_CAD,CURRENCY,NZD/CAD,-4,0.03
2,EUR_CHF,CURRENCY,EUR/CHF,-4,0.04
3,SGD_CHF,CURRENCY,SGD/CHF,-4,0.05
4,EUR_CZK,CURRENCY,EUR/CZK,-4,0.05
...,...,...,...,...,...
63,USD_SEK,CURRENCY,USD/SEK,-4,0.03
64,GBP_SGD,CURRENCY,GBP/SGD,-4,0.05
65,EUR_PLN,CURRENCY,EUR/PLN,-4,0.05
66,HKD_JPY,CURRENCY,HKD/JPY,-4,0.1


In [16]:
ins_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [17]:
ins_df.columns


Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [18]:
ins_df.name

0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object

In [19]:
ins_df.name.unique

<bound method Series.unique of 0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object>